In [1]:
# @title Import Libraries
import sys
import glob, os
import cv2
import numpy as np
import pandas as pd
import geopy.distance
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from skimage import exposure
from skimage import feature
import skimage.measure


2024-07-31 16:20:20.237192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 16:20:21.215619: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2024-07-31 16:20:21.215719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdjkhosh/.local/lib/python3.10/sit

In [2]:
# Setup paths and import local libraries
%cd ../../

prj_path = Path('SkyAI-Sim')
sys.path.insert(0, str(prj_path))
from src.utils import img_helper
from src.utils.io_helper import str_to_floats
from src.utils import consts
from src.data.skyai import SkyAI

args = consts.ARGS
args

/home/sdjkhosh/Projects/SkyAI-Sim


usage: SkyAI Sim Project [-h] [--coords COORDS] [--fov FOV]
                         [--aspect_ratio ASPECT_RATIO [ASPECT_RATIO ...]]
                         [--utm UTM] [--data_dir DATA_DIR] [--vmargin VMARGIN]
                         [--img_size IMG_SIZE [IMG_SIZE ...]]
                         [--overlap OVERLAP] [--batch_size BATCH_SIZE]
                         [--seed SEED]
SkyAI Sim Project: error: argument --fov: invalid float value: '/home/sdjkhosh/.local/share/jupyter/runtime/kernel-v2-3614848YDxX8HOcCzMU.json'


Namespace(coords=[35.16, -89.9, 35.115, -89.823, 400.0], fov=78.8, aspect_ratio=[4, 3], utm='EPSG:32616', data_dir='dataset/Memphis/', vmargin=20, img_size=[400, 400, 3], overlap=0, batch_size=8, seed=2024)

In [ ]:
aerial_data = SkyAI(
        args=args,
        map_type='roadmap',
        data_dir=args.data_dir,
        overlap=args.overlap
        )
aerial_data.config(download_raster=True)


In [ ]:
aerial_imgs_path = glob.glob(os.path.join(aerial_data.data_dir / 'images', "*.jpg"))
aerial_imgs_path.sort()
print(len(aerial_imgs_path))
aerial_imgs, aerial_titles = img_helper.choose_random_images(nx*ny,
                                                            aerial_imgs_path,
                                                            output_dir=aerial_data.data_dir / 'random_sample',
                                                            obj=aerial_data,
                                                            seed=seed)

In [ ]:
import skimage.measure
entropys = [skimage.measure.shannon_entropy(img) for img in aerial_imgs]

img_helper.plot_multy(aerial_imgs, aerial_data.data_dir /'entropies', nx, ny, entropys)

In [ ]:
road_n = [cv2.normalize(img, None, alpha = 0, beta = 255,
                       norm_type = cv2.NORM_MINMAX,
                       dtype = cv2.CV_32F).astype('uint8') for img in aerial_imgs]
road_gray = [cv2.cvtColor(real_img, cv2.COLOR_BGR2GRAY) for real_img in road_n]
img_helper.plot_multy(road_gray, '../Gray Scale \nRoadmap Images', nx, ny, aerial_titles)

In [ ]:

road_eq = [cv2.equalizeHist(img) for img in road_gray]
img_helper.plot_multy(road_eq, '../Equalized Gray Scale \nRoadmap Images', nx, ny, aerial_titles)

In [ ]:
# equalize each color channel
eq_color = [cv2.merge([cv2.equalizeHist(channel) for channel in cv2.split(img)]) for img in road_n]
img_helper.plot_multy(eq_color, '../Equalized Color \nRoadmap Images', nx, ny, aerial_titles)